# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 8:30AM,255030,10,0086298627,ISDIN Corporation,Released
1,Jan 16 2023 8:30AM,255031,10,0086298626,ISDIN Corporation,Released
2,Jan 16 2023 8:30AM,255030,10,0086298630,ISDIN Corporation,Released
3,Jan 16 2023 8:30AM,255030,10,0086298631,ISDIN Corporation,Released
4,Jan 16 2023 8:30AM,255030,10,0086298632,ISDIN Corporation,Released
5,Jan 16 2023 8:30AM,255030,10,0086298633,ISDIN Corporation,Released
6,Jan 16 2023 8:30AM,255030,10,0086298628,ISDIN Corporation,Released
7,Jan 16 2023 8:30AM,255030,10,0086298629,ISDIN Corporation,Released
8,Jan 16 2023 8:30AM,255032,10,Enova-11206,Emerginnova,Released
9,Jan 13 2023 3:12PM,255033,19,7467297,"NAPP Technologies, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,255029,Released,29
13,255030,Released,7
14,255031,Released,1
15,255032,Released,1
16,255033,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
255029,NaN,29.0
255030,NaN,7.0
255031,NaN,1.0
255032,NaN,1.0
255033,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254992,0.0,8.0
254995,0.0,1.0
254996,2.0,3.0
255015,0.0,1.0
255021,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254992,0,8
254995,0,1
254996,2,3
255015,0,1
255021,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254992,0,8
1,254995,0,1
2,254996,2,3
3,255015,0,1
4,255021,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254992,,8
1,254995,,1
2,254996,2,3
3,255015,,1
4,255021,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 8:30AM,255030,10,ISDIN Corporation
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation
8,Jan 16 2023 8:30AM,255032,10,Emerginnova
9,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC"
10,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation
39,Jan 13 2023 2:29PM,255028,10,ISDIN Corporation
48,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC"
83,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc."
104,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC"
105,Jan 13 2023 1:13PM,255021,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 16 2023 8:30AM,255030,10,ISDIN Corporation,,7
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,,1
2,Jan 16 2023 8:30AM,255032,10,Emerginnova,,1
3,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",,1
4,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation,,29
5,Jan 13 2023 2:29PM,255028,10,ISDIN Corporation,4,5
6,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",,35
7,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",1,20
8,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC",,1
9,Jan 13 2023 1:13PM,255021,19,"GUSA Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 8:30AM,255030,10,ISDIN Corporation,7,
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,1,
2,Jan 16 2023 8:30AM,255032,10,Emerginnova,1,
3,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",1,
4,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation,29,
5,Jan 13 2023 2:29PM,255028,10,ISDIN Corporation,5,4
6,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",35,
7,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",20,1
8,Jan 13 2023 1:33PM,255023,10,"Nextsource Biotechnology, LLC",1,
9,Jan 13 2023 1:13PM,255021,19,"GUSA Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 8:30AM,255030,10,ISDIN Corporation,7,
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,1,
2,Jan 16 2023 8:30AM,255032,10,Emerginnova,1,
3,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",1,
4,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation,29,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 8:30AM,255030,10,ISDIN Corporation,7,NaN
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,1,NaN
2,Jan 16 2023 8:30AM,255032,10,Emerginnova,1,NaN
3,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",1,NaN
4,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation,29,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 8:30AM,255030,10,ISDIN Corporation,7,0.0
1,Jan 16 2023 8:30AM,255031,10,ISDIN Corporation,1,0.0
2,Jan 16 2023 8:30AM,255032,10,Emerginnova,1,0.0
3,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",1,0.0
4,Jan 13 2023 2:52PM,255029,10,ISDIN Corporation,29,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1530173,44,4.0
15,764983,12,2.0
18,255015,1,0.0
19,765081,37,0.0
20,255026,20,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1530173,44,4.0
1,15,764983,12,2.0
2,18,255015,1,0.0
3,19,765081,37,0.0
4,20,255026,20,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,44,4.0
1,15,12,2.0
2,18,1,0.0
3,19,37,0.0
4,20,20,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,15,Released,12.0
2,18,Released,1.0
3,19,Released,37.0
4,20,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20
Status,,,,,
Executing,4.0,2.0,0.0,0.0,1.0
Released,44.0,12.0,1.0,37.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20
0,Executing,4.0,2.0,0.0,0.0,1.0
1,Released,44.0,12.0,1.0,37.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20
0,Executing,4.0,2.0,0.0,0.0,1.0
1,Released,44.0,12.0,1.0,37.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()